In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

In [3]:
# TODO: Implement FashionDataset once we know what the dataset looks like
class FashionDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()

    def __len__(self):
        pass

    def __getitem__(self, index):
        pass

In [ ]:
def train(model: nn.Module, data: DataLoader, loss_fcn: nn.Module, epochs: int = 10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        running_loss = 0.0

        for anchor, positive, negative in tqdm(data, f"Epoch {epoch}", unit="batch"):
            optimizer.zero_grad()

            # Send images to the device
            anchor.to(model.device)
            positive.to(model.device)
            negative.to(model.device)

            # Forward pass
            anchor_features = model(anchor)
            positive_features = model(positive)
            negative_features = model(negative)

            # Compute the loss
            loss = loss_fcn(anchor_features, positive_features, negative_features)

            running_loss += loss.item()

            # Backward pass
            loss.backward()
            optimizer.step()
        
        print(f"Epoch {epoch} Loss: {running_loss / len(data)}")

In [ ]:
# Load the dataset
dataset = FashionDataset()

# Define the dataloader
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model
model = models.resnet50()

In [ ]:
train(model, train_loader, nn.TripletMarginLoss(), epochs=100)